In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [3]:
import os
import random
import time
import itertools  
import numpy                 as np
import tensorflow            as tf
import tensorflow_addons     as tfa
import matplotlib.pyplot     as plt
import pandas                as pd
import gudhi                 as gd
import gudhi.representations as sktda
import sys

from difftda                              import *
from gudhi.representations.vector_methods import Atol as atol
from gudhi.representations.kernel_methods import SlicedWassersteinKernel as swk
from gudhi.wasserstein                    import wasserstein_distance
from gudhi.representations                import pairwise_persistence_diagram_distances as ppdd
from mpl_toolkits.mplot3d                 import Axes3D
from scipy.linalg                         import expm
from scipy.io                             import loadmat
from scipy.sparse                         import csgraph
from scipy.linalg                         import eigh
from sklearn.base                         import BaseEstimator, TransformerMixin
from sklearn.metrics                      import pairwise_distances, accuracy_score
from sklearn.manifold                     import MDS, LocallyLinearEmbedding, SpectralEmbedding
from sklearn.preprocessing                import MinMaxScaler, Normalizer, LabelEncoder
from sklearn.pipeline                     import Pipeline, FeatureUnion
from sklearn.svm                          import SVC
from sklearn.ensemble                     import RandomForestClassifier
from sklearn.neighbors                    import KNeighborsClassifier
from sklearn.model_selection              import GridSearchCV, KFold, StratifiedKFold
from sklearn.cluster                      import KMeans

/Users/nyckeesluca/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [123]:
from tqdm import tqdm

# Small example

In [4]:
np.random.seed(1)
angles = np.random.uniform(0,2*np.pi,100)
X = np.hstack([ np.cos(angles)[:,None], np.sin(angles)[:,None] ])
dim = 1

In [ ]:
#X = np.array([[0.1,0.],[1.5,1.6]])
X = np.array([[0.1,0.],[1.5,1.5],[0.,1.6]])
dim = 0

In [7]:
class IndexRipsModel(tf.keras.Model):
    def __init__(self, X, mel=10, dim=dim, card=50):
        super(IndexRipsModel, self).__init__()
        self.X = X
        self.mel = mel
        self.dim = dim
        self.card = card
        
    def call(self, D):
        m, d, c = self.mel, self.dim, self.card
        
        # Compute distance matrix
        DX = tfa.losses.metric_learning.pairwise_distance(self.X)
        DXX = tf.reshape(DX, [1, DX.shape[0], DX.shape[1]])
        
        # Turn numpy function into tensorflow function
        RipsTF = lambda DX: tf.numpy_function(Rips, [DX, m, d, c], [tf.int32 for _ in range(4*c)])
        
        # Compute vertices associated to positive and negative simplices 
        # Don't compute gradient for this operation
        ids = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(RipsTF,DXX,dtype=[tf.int32 for _ in range(4*c)]))
        
        # Get persistence diagram by simply picking the corresponding entries in the distance matrix
        dgm = tf.reshape(tf.gather_nd(D, tf.reshape(ids, [2*c,2])), [c,2])
        return dgm

In [8]:
XTF = tf.Variable(X, tf.float32)
lr = 1
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

num_epochs = 1
losses, Dgs, Xs, grads = [], [], [], []
for epoch in range(num_epochs+1):
        
    with tf.GradientTape() as tape:
        
        Dg = RipsModel(X=XTF, mel=10, dim=dim, card=10).call()
        loss = -wasserstein_distance(Dg, tf.constant(np.empty([0,2])), order=1, enable_autodiff=True)
        
    Dgs.append(Dg.numpy())            
    Xs.append(XTF.numpy())
    losses.append(loss.numpy())
    
    gradients = tape.gradient(loss, [XTF])
    grads.append(gradients[0].numpy())
    optimizer.apply_gradients(zip(gradients, [XTF]))

2022-04-20 16:17:38.371006: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-20 16:17:38.427319: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fab251441a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-04-20 16:17:38.427335: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [9]:
pts_to_move = np.argwhere(np.linalg.norm(grads[0], axis=1) != 0).ravel()
plt.figure()
for pt in pts_to_move:
    plt.arrow(Xs[0][pt,0], Xs[0][pt,1], -lr*grads[0][pt,0], -lr*grads[0][pt,1], color='blue',
              length_includes_head=True, head_length=.05, head_width=.1, zorder=10)
plt.scatter(Xs[0][:,0], Xs[0][:,1], c='red', s=50, alpha=.2,  zorder=3)
plt.scatter(Xs[0][pts_to_move,0], Xs[0][pts_to_move,1], c='red',   s=150, marker='o', zorder=2, alpha=.7, label='Step i')
plt.scatter(Xs[1][pts_to_move,0], Xs[1][pts_to_move,1], c='green', s=150, marker='o', zorder=1, alpha=.7, label='Step i+1')
plt.axis('square')
#plt.xlim([-.7,2.3])
#plt.ylim([-.7,2.3])
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

# Dimension Reduction

In this section, we implement the dimension reduction experiment, where we improve dimension reduction with autoencoders using 1-dimensional homology. Use `use_reg=True` if you want to add a topological loss to the autoencoder, and `common_topo_autoencoder=True` if you want to use the topological loss of Moor et al (https://arxiv.org/abs/1906.00722).

In [ ]:
use_reg = True
common_topo_autoencoder = False

In [ ]:
N = 100
eps = .1
Rell1x, Rell1y = 1, 1
theta1x, theta1y = 0, 0
Rell2x, Rell2y = .8, .6
theta2x, theta2y = -np.pi/4, -np.pi/6

np.random.seed(100)

angles = np.linspace(0,2*np.pi,N)[:,np.newaxis]
Rcic = 1.
cic = np.hstack([np.zeros([N,1]), 
                 Rcic * np.cos(angles) + eps * np.random.uniform(-1,1,(N,1)),
                 Rcic * np.sin(angles) + eps * np.random.uniform(-1,1,(N,1))])
ell1 = np.hstack([
    Rell1x * np.cos(angles) + eps * np.random.uniform(-1,1,(N,1)),
    Rell1y * np.sin(angles) + eps * np.random.uniform(-1,1,(N,1)),
    np.zeros([N,1])])
R1x = np.array([[1,0,0],[0,np.cos(theta1x),-np.sin(theta1x)],[0,np.sin(theta1x),np.cos(theta1x)]])
R1y = np.array([[np.cos(theta1y),0,np.sin(theta1y)],[0,1,0],[-np.sin(theta1y),0,np.cos(theta1y)]])
ell2 = np.hstack([
    Rell2x * np.cos(angles) + eps * np.random.uniform(-1,1,(N,1)),
    Rell2y * np.sin(angles) + eps * np.random.uniform(-1,1,(N,1)),
    np.zeros([N,1])])
R2x = np.array([[1,0,0],[0,np.cos(theta2x),-np.sin(theta2x)],[0,np.sin(theta2x),np.cos(theta2x)]])
R2y = np.array([[np.cos(theta2y),0,np.sin(theta2y)],[0,1,0],[-np.sin(theta2y),0,np.cos(theta2y)]])
ell1 = ell1 + np.array([[.3,.3,0.]])
ell2 = ell2 + np.array([[-.1,-.1,0.]])
Xinit = np.vstack([cic, cic[0,:] + np.dot(np.dot(ell1,R1x),R1y)]) #, cic[int(N/2),:] + np.dot(np.dot(ell2,R2x),R2y)])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(Xinit[:,0], Xinit[:,1], Xinit[:,2], 
                c=np.concatenate([np.ones([N]),2*np.ones([N])]), #3*np.ones([N])]), 
                s=5, cmap='rainbow')
x_limits, y_limits, z_limits = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
x_range, x_middle = abs(x_limits[1] - x_limits[0]), np.mean(x_limits)
y_range, y_middle = abs(y_limits[1] - y_limits[0]), np.mean(y_limits)
z_range, z_middle = abs(z_limits[1] - z_limits[0]), np.mean(z_limits)
plot_radius = 0.5*max([x_range, y_range, z_range])
ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
ax.grid(False)
plt.title('Initial point cloud')
plt.savefig('dimredinit_' + str(use_reg) + '.png')

In [ ]:
Rot = []
for pt in Xinit:
    M = np.zeros((3,3))
    M[0,1] = pt[0]
    M[0,2] = pt[1]
    M[1,0] = -pt[0]
    M[2,0] = -pt[1]
    M[1,2] = pt[2]
    M[2,1] = -pt[2]
    Rot.append(expm(M).flatten())
Xinit = np.asarray(Rot)
Dinit = pairwise_distances(Xinit)

In [ ]:
class IndexRipsModel(tf.keras.Model):
    def __init__(self, X, mel=12, dim=1, card=50):
        super(IndexRipsModel, self).__init__()
        self.X = X
        self.mel = mel
        self.dim = dim
        self.card = card
        
    def call(self, D):
        m, d, c = self.mel, self.dim, self.card
        
        # Compute distance matrix
        DX = tfa.losses.metric_learning.pairwise_distance(self.X)
        DXX = tf.reshape(DX, [1, DX.shape[0], DX.shape[1]])
        
        # Turn numpy function into tensorflow function
        RipsTF = lambda DX: tf.numpy_function(Rips, [DX, m, d, c], [tf.int32 for _ in range(4*c)])
        
        # Compute vertices associated to positive and negative simplices 
        # Don't compute gradient for this operation
        ids = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(RipsTF,DXX,dtype=[tf.int32 for _ in range(4*c)]))
        
        # Get persistence diagram by simply picking the corresponding entries in the distance matrix
        dgm = tf.reshape(tf.gather_nd(D, tf.reshape(ids, [2*c,2])), [c,2])
        return dgm

In [ ]:
st = gd.RipsComplex(distance_matrix=Dinit, max_edge_length=2).create_simplex_tree(max_dimension=2)
st.persistence()
Dg1init = st.persistence_intervals_in_dimension(1)

initial_learning_rate = 0.05
lr = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=1e5, decay_rate=0.99)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
Dg1iTF = tf.constant(np.array(Dg1init,dtype=np.float32))

seed = 0
tf.random.set_seed(seed)

autoencoder = tf.keras.models.Sequential()
autoencoder.add(tf.keras.Input(shape=(Xinit.shape[1],)))
autoencoder.add(tf.keras.layers.Dense(32, activation='relu'))
autoencoder.add(tf.keras.layers.BatchNormalization())
autoencoder.add(tf.keras.layers.Dense(32, activation='relu'))
autoencoder.add(tf.keras.layers.BatchNormalization())
autoencoder.add(tf.keras.layers.Dense(2, activation=None))
autoencoder.add(tf.keras.layers.Dense(32, activation='relu'))
autoencoder.add(tf.keras.layers.BatchNormalization())
autoencoder.add(tf.keras.layers.Dense(32, activation='relu'))
autoencoder.add(tf.keras.layers.BatchNormalization())
autoencoder.add(tf.keras.layers.Dense(Xinit.shape[1], activation=None))

reduced_data = tf.keras.Model(autoencoder.input, autoencoder.get_layer(index=4).output) 

Xred = reduced_data.predict(Xinit)
st = gd.RipsComplex(Xred, max_edge_length=2).create_simplex_tree(max_dimension=2)
st.persistence()
Dg1red = st.persistence_intervals_in_dimension(1)

a3 = 1
a2 = 1e1 if use_reg else 0
num_epochs = 200
losses, l2s, l3s, Dgs1, Xs = [], [], [], [], []
for epoch in range(num_epochs+1):
        
    with tf.GradientTape() as tape:
        
        XXinit = autoencoder(Xinit)
        Xred = reduced_data(Xinit)

        if use_reg and epoch >= 0:
            if common_topo_autoencoder:
                XinitTF = tf.convert_to_tensor(Xinit, dtype=tf.float32)
                DZ = tfa.losses.metric_learning.pairwise_distance(Xred)
                DX = tfa.losses.metric_learning.pairwise_distance(XinitTF)
                DZX = IndexRipsModel(X=Xred, mel=3.,  dim=0, card=50).call(DX)
                DZZ = IndexRipsModel(X=Xred, mel=3.,  dim=0, card=50).call(DZ)
                DXX = IndexRipsModel(X=XinitTF, mel=3., dim=0, card=50).call(DX)
                DXZ = IndexRipsModel(X=XinitTF, mel=3., dim=0, card=50).call(DZ)
            else:
                Dg1 = RipsModel(X=Xred, mel=3., dim=1, card=50).call()
                Dgs1.append(Dg1.numpy())
            
        Xs.append(Xred)
        l3 = a3*tf.math.reduce_sum(tf.square(Xinit-XXinit))

        if use_reg and epoch >= 0:
            if common_topo_autoencoder:
                l2 = tf.math.reduce_sum(tf.square(DXX-DXZ)) + tf.math.reduce_sum(tf.square(DZX-DZZ))
            else:
                l2 = a2*wasserstein_distance(Dg1, Dg1iTF, order=1, enable_autodiff=True)
                
        loss = l3

        if use_reg and epoch >= 0:
            loss = loss + l2
    
    gradients = tape.gradient(loss, autoencoder.trainable_variables)
    tf.random.set_seed(epoch)
    optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))
    
    losses.append(loss.numpy())
    if use_reg and epoch >= 0:
        l2s.append(l2.numpy())
    l3s.append(l3.numpy())

In [ ]:
plt.figure()
plt.plot(losses)
plt.title('Losses')
plt.savefig('dimredloss_' + str(use_reg) + '.png')

In [ ]:
plt.figure()
plt.scatter(Xs[-1][:,0], Xs[-1][:,1], s=10, c=np.concatenate([np.ones([N]),2*np.ones([N])]), cmap='rainbow')
plt.title('LowD point cloud at epoch ' + str(epoch))
plt.savefig('dimredafter_' + str(use_reg) + '.png')

In [ ]:
if use_reg and not common_topo_autoencoder:

    plt.figure()
    plt.scatter(Dg1init[:,0],  Dg1init[:,1],  label='initial')
    plt.scatter(Dg1red[:,0],   Dg1red[:,1],   label='lowD')
    plt.scatter(Dgs1[-1][:,0], Dgs1[-1][:,1], label='trained lowD')
    plt.plot([0,1.5],[0,1.5])
    plt.axis('square')
    plt.title('Dim. 1 PD')
    plt.legend()
    plt.savefig('dimreddg10_' + str(use_reg) + '.png')

    plt.figure()
    plt.scatter(Dg1init[:,0],  Dg1init[:,1],  label='initial')
    plt.scatter(Dg1red[:,0],   Dg1red[:,1],   label='lowD')
    for D in Dgs1[0:-1]:
        plt.scatter(D[:,0], D[:,1], s=20, marker='D', alpha=0.1)
    plt.scatter(Dgs1[-1][:,0], Dgs1[-1][:,1], s=40, marker='D', c='green', label='trained lowD')
    plt.plot([0,1.5],[0,1.5])
    plt.axis('square')
    plt.title('Dim. 1 PD')
    plt.legend()
    plt.savefig('dimreddg11_' + str(use_reg) + '.png')

# Regression

In this section, we implement the linear regression experiment, where we recover hidden coefficients using 0-dimensional homology.

In [ ]:
np.random.seed(0)
n, p = 50, 100
betastar = np.concatenate([np.linspace(-1.,1.,33) for _ in range(3)] + [[-1.]])
X = np.random.multivariate_normal(mean=np.zeros(shape=[p]), cov=np.eye(p), size=n)

Y = np.matmul(X, betastar) + .05 * np.random.randn(n)
X, Y = np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)
stbase = gd.SimplexTree()
for i in range(p-1):
    stbase.insert([i,i+1], -1e10)
f = open('data/beta_simplextree.txt', 'w')
for (s,_) in stbase.get_filtration():
    for v in s:
        f.write(str(v) + " ")
    f.write("\n")
f.close()

betainit = np.random.uniform(low=-1., high=1., size=[p])
betainit[np.array([25,60,99])] = np.array([-1,-1,-1])

In [ ]:
plt.figure()
plt.plot(betastar)
plt.title('Ground-truth coefficients')
plt.savefig('reggt.png')

In [ ]:
plt.figure()
plt.plot(betainit)
plt.title('Coefficients at epoch 0')
plt.savefig('reginit.png')

In [ ]:
beta = tf.Variable(initial_value=np.array(betainit[:,np.newaxis], dtype=np.float32), trainable=True)
model = SimplexTreeModel(beta, stbase='data/beta_simplextree.txt', dim=0, card=100)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-5, decay_steps=10, decay_rate=.001)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms, betas = [], [], []
alpha, gamma, delta = 1, 1e4, 1e3
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        dgm = model.call()
        loss = alpha * tf.reduce_sum(tf.square(tf.matmul(X, -beta) - Y)) \
             + gamma * tf.reduce_sum(tf.abs(dgm[2:,1]-dgm[2:,0])) \
             + delta * tf.reduce_sum(tf.abs(beta[1:]-beta[:-1]))
             
    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append(dgm)
    betas.append(beta.numpy()[:,0])

beta_stdtop = -betas[-1]

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('regloss.png')

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker='D', c='blue')
for dg in dgms[:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='red')
plt.plot([-1,1], [-1,1])
plt.title('Optimized persistence diagrams')
plt.savefig('regdg.png')

In [ ]:
beta = tf.Variable(initial_value=np.array(betainit[:,np.newaxis], dtype=np.float32), trainable=True)
model = SimplexTreeModel(beta, stbase='data/beta_simplextree.txt', dim=0, card=100)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-5, decay_steps=10, decay_rate=.001)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

In [ ]:
losses, dgms, betas = [], [], []
alpha, gamma, delta = 1, 0, 1e3
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        dgm = model.call()
        loss = alpha * tf.reduce_sum(tf.square(tf.matmul(X, -beta) - Y)) \
             + gamma * tf.reduce_sum(tf.abs(dgm[2:,1]-dgm[2:,0])) \
             + delta * tf.reduce_sum(tf.abs(beta[1:]-beta[:-1]))
             
    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append(dgm)
    betas.append(beta.numpy()[:,0])
                 
beta_stdtot = -betas[-1]

beta = tf.Variable(initial_value=np.array(betainit[:,np.newaxis], dtype=np.float32), trainable=True)
model = SimplexTreeModel(beta, stbase='data/beta_simplextree.txt', dim=0, card=100)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-5, decay_steps=10, decay_rate=.001)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms, betas = [], [], []
alpha, gamma, delta = 2, 0, 0
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        dgm = model.call()
        loss = alpha * tf.reduce_sum(tf.square(tf.matmul(X, -beta) - Y)) \
             + gamma * tf.reduce_sum(tf.abs(dgm[2:,1]-dgm[2:,0])) \
             + delta * tf.reduce_sum(tf.abs(beta[1:]-beta[:-1]))
                     
    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append(dgm)
    betas.append(beta.numpy()[:,0])
    
beta_std = -betas[-1]

In [ ]:
plt.figure()
plt.plot(betastar, label='ground-truth')
plt.plot(beta_std, label='MSE')
plt.plot(beta_stdtot, label='MSE+TV')
plt.plot(beta_stdtop, label='MSE+TV+TOP')
plt.legend(loc='upper left')
plt.savefig('regafter.png')

In [ ]:
MSEstd, MSEtop, MSEtot = [], [], []
for s in range(1000):
    np.random.seed(s)
    Xnew = np.random.multivariate_normal(mean=np.zeros(shape=[p]), cov=np.eye(p), size=n)
    Ynew = np.matmul(Xnew, betastar)
    mse_std = np.square(np.matmul(Xnew, beta_std) - Ynew).sum()
    mse_tot = np.square(np.matmul(Xnew, beta_stdtot) - Ynew).sum()
    mse_top = np.square(np.matmul(Xnew, beta_stdtop) - Ynew).sum()
    MSEstd.append(mse_std)
    MSEtot.append(mse_tot)
    MSEtop.append(mse_top)
    
plt.figure()
plt.boxplot([MSEstd, MSEtot, MSEtop], labels=['MSE', 'MSE+TV', 'MSE+TV+TOP'])
plt.title('MSE on random test sets')
plt.savefig('regmse.png')

# Image

In this section, we implement the image experiment, where we remove the noise of an image using 0-dimensional homology. Use `use_reg=True` if you want to use a topological loss.

In [110]:
use_reg = 1

In [111]:
I = np.array(pd.read_csv('data/mnist_test.csv', header=None, sep=','), dtype=np.float32)
idx = np.argwhere(I[:,0] == 8)
image = np.reshape(-I[idx[8],1:], [28,28])
image = (image-image.min())/(image.max()-image.min())
image_clean = np.array(image)
image[2:5,2:5]        -= 0.6
image[25:27,25:27]    -= 0.6
image[25:27,2:5]      -= 0.6
image[1:4,24:26]      -= 0.6

In [112]:
plt.figure()
plt.imshow(image, cmap='Greys')
plt.title('Image at epoch 0')
plt.savefig('imbefore_' + str(use_reg) + '.png')

<IPython.core.display.Javascript object>

In [125]:
X = tf.Variable(initial_value=np.array(image, dtype=np.float32), trainable=True)
model = CubicalModel(X, dim=0, card=100)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-3, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms, empty = [], [], np.empty([0,2])
alpha = 10.
gamma = 1. if use_reg else 0
for epoch in tqdm(range(100+1)):
    
    with tf.GradientTape() as tape:
        
        dgm = model.call()
        if use_reg:
            #loss = alpha * tf.math.reduce_sum(tf.abs(dgm[:,1]-dgm[:,0])) + \
            loss = gamma * tf.math.reduce_sum(tf.math.minimum(tf.abs(model.X), tf.abs(1.-model.X)))
        else:
            loss = alpha * tf.math.reduce_sum(tf.abs(dgm[:,1]-dgm[:,0]))

    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append(dgm)

100%|██████████| 101/101 [00:09<00:00, 11.18it/s]


In [ ]:
plt.figure()
plt.imshow(model.X.numpy(), cmap='Greys')
plt.title('Image at epoch ' + str(epoch))
plt.savefig('imafter_' + str(use_reg) + '.png')

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('imloss_' + str(use_reg) + '.png')

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker='D', c='blue')
for dg in dgms[:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=0.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='red')
plt.plot([0,1], [0,1])
plt.title('Optimized persistence diagrams')
plt.savefig('imdg_' + str(use_reg) + '.png')

# Point cloud

In this section, we implement the point cloud experiment, where we optimize loops in a point cloud using 1-dimensional homology.

In [19]:
use_reg = 1

In [20]:
np.random.seed(1)
Xinit = np.array(np.random.uniform(high=1., low=-1., size=(300,2)), dtype=np.float32)

In [21]:
plt.figure()
plt.scatter(Xinit[:,0], Xinit[:,1])
plt.title('Point cloud at epoch 0')
plt.savefig('pcinit_' + str(use_reg) + '.png')

<IPython.core.display.Javascript object>

In [23]:
X = tf.Variable(initial_value=Xinit, trainable=True)
model = RipsModel(X=X, mel=12., dim=1, card=50)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-1, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.)
sigma = 0.001

losses, dgms = [], []
for epoch in range(5+1):
    
    with tf.GradientTape() as tape:
        
        dgm = model.call()
        if use_reg:
            loss = -tf.math.reduce_sum(tf.square(.5*(dgm[:,1]-dgm[:,0]))) + tf.reduce_sum(tf.maximum(tf.abs(X)-1, 0))
        else:
            loss = -tf.math.reduce_sum(tf.square(.5*(dgm[:,1]-dgm[:,0])))

    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append(dgm)

In [16]:
plt.figure()
plt.scatter(model.X.numpy()[:,0], model.X.numpy()[:,1])
plt.title('Point cloud at epoch ' + str(epoch))
plt.savefig('pcafter_' + str(use_reg) + '.png')

<IPython.core.display.Javascript object>

In [17]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('pcloss_' + str(use_reg) + '.png')

<IPython.core.display.Javascript object>

In [18]:
plt.figure()
for dg in dgms[:5:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=0.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='green')
plt.plot([-0.,.2], [-0.,.2])
plt.title('Optimized persistence diagrams')
plt.savefig('pcdg_' + str(use_reg) + '.png')

<IPython.core.display.Javascript object>

# Noisy point cloud

In this section, we implement the noisy point cloud experiment, where we optimize the connected components of a noisy point cloud using 0-dimensional homology.

In [210]:
n, epsilon, nout = 100, .2, 3
x, y = np.cos(np.linspace(0,2*np.pi,n)), np.sin(np.linspace(0,2*np.pi,n))
np.random.seed(10)
ex, ey = np.random.uniform(low=-epsilon,high=epsilon,size=n), np.random.uniform(low=-epsilon,high=epsilon,size=n)
outliers = np.random.uniform(low=-.7, high=.7, size=(nout,2))

In [217]:
fig = plt.figure()
plt.scatter(x+ex, y+ey)
plt.scatter(outliers[:,0], outliers[:,1])
plt.title('Point cloud at epoch 0')
plt.show()
#plt.savefig('noisypcinit.png')

<IPython.core.display.Javascript object>

In [ ]:
st = gd.RipsComplex(distance_matrix=pairwise_distances(np.hstack([x[:,np.newaxis],y[:,np.newaxis]])), max_edge_length=2.).create_simplex_tree(max_dimension=2)
st.persistence()
D = np.array(st.persistence_intervals_in_dimension(0), dtype=np.float32)[:-1]

In [ ]:
Xinit = np.array(np.vstack([np.hstack([(x+ex)[:,np.newaxis], (y+ey)[:,np.newaxis]]),outliers]), dtype=np.float32)

X = tf.Variable(initial_value=Xinit, trainable=True)
model = RipsModel(X=X, mel=2., dim=0, card=150)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-1, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms = [], []
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        dgm = model.call()
        loss = tf.square(wasserstein_distance(dgm, tf.constant(D), order=2, enable_autodiff=True))
        
    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append(dgm)        

In [ ]:
plt.figure()
plt.scatter(model.X.numpy()[:,0], model.X.numpy()[:,1])
plt.title('Point cloud at epoch ' + str(epoch))
plt.savefig('noisypcafter.png')

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('noisypcloss.png')

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker='D', c='blue')
for dg in dgms[:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='red')
plt.plot([-1,1], [-1,1])
plt.title('Optimized persistence diagrams')
plt.savefig('noisypcdg.png')

# 3D shape

In this section, we implement the 3D shape experiment, where we optimize the values on a 3D shape using 0-dimensional homology.

In [218]:
faces, coord = np.loadtxt('data/human_faces', dtype=float)[:,1:], np.loadtxt('data/human_coords', dtype=float)
stbase = gd.SimplexTree()
for i in range(len(faces)):
    stbase.insert(faces[i,:], -1e10)
f = open('data/human_simplextree.txt', 'w')
for (s,_) in stbase.get_filtration():
    for v in s:
        f.write(str(v) + " ")
    f.write('\n')
f.close()
Finit = coord[:,2]

In [226]:
coord.shape

(4706, 3)

In [225]:
Finit.shape

(4706,)

In [219]:
step = 1
fig = plt.figure()
cm = plt.cm.get_cmap('rainbow')
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(coord[::step,0], coord[::step,1], coord[::step,2], c=Finit[::step], s=2, 
                vmin=0, vmax=.75, cmap=cm)
x_limits, y_limits, z_limits = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
x_range, x_middle = abs(x_limits[1] - x_limits[0]), np.mean(x_limits)
y_range, y_middle = abs(y_limits[1] - y_limits[0]), np.mean(y_limits)
z_range, z_middle = abs(z_limits[1] - z_limits[0]), np.mean(z_limits)
plot_radius = 0.5*max([x_range, y_range, z_range])
ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
plt.title('Function at epoch 0')
plt.savefig('d3sinit.png')

<IPython.core.display.Javascript object>

In [224]:
F = tf.Variable(initial_value=np.array(Finit, dtype=np.float32), trainable=True)
model = SimplexTreeModel(F, stbase='data/human_simplextree.txt', dim=0, card=50)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-1, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms = [], []
alpha, gamma = 1., .001
for epoch in tqdm(range(100+1)):
    
    with tf.GradientTape() as tape:
        
        dgm = model.call()
        loss = alpha * tf.square(wasserstein_distance(dgm, tf.constant(np.array([[-.98,-.03]], dtype=np.float32)), order=2, enable_autodiff=True))               
    
    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    gradients = [tf.convert_to_tensor(gradients[0]) + np.random.normal(loc=0., scale=sigma, size=gradients[0].dense_shape)]
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append(dgm)

100%|██████████| 101/101 [00:12<00:00,  7.98it/s]


In [18]:
step = 1
fig = plt.figure()
cm = plt.cm.get_cmap('rainbow')
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(coord[::step,0], coord[::step,1], coord[::step,2], c=F.numpy()[::step], s=1, 
                vmin=0, vmax=.75, cmap=cm)
x_limits, y_limits, z_limits = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
x_range, x_middle = abs(x_limits[1] - x_limits[0]), np.mean(x_limits)
y_range, y_middle = abs(y_limits[1] - y_limits[0]), np.mean(y_limits)
z_range, z_middle = abs(z_limits[1] - z_limits[0]), np.mean(z_limits)
plot_radius = 0.5*max([x_range, y_range, z_range])
ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
plt.title('Function at epoch ' + str(epoch))
plt.savefig('d3safter.png')

<IPython.core.display.Javascript object>

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('d3sloss.png')

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker='D', c='blue')
for dg in dgms[:-1:2]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker='D', alpha=.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker='D', c='red')
plt.plot([-1,1], [-1,1])
plt.title('Optimized persistence diagrams')
plt.savefig('d3sdg.png')

# Integral sheaf metric

## Experiments

### Lower-star filtration on simplicial complex

In [90]:
faces, coord = np.loadtxt('data/human_faces', dtype=float)[:,1:], np.loadtxt('data/human_coords', dtype=float)
stbase = gd.SimplexTree()
for i in range(len(faces)):
    stbase.insert(faces[i,:], -1e10)
f = open('data/human_simplextree.txt', 'w')
for (s,_) in stbase.get_filtration():
    for v in s:
        f.write(str(v) + " ")
    f.write('\n')
f.close()
Finit = coord[:,2]

In [10]:
step = 1
fig = plt.figure()
cm = plt.cm.get_cmap('rainbow')
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(coord[::step,0], coord[::step,1], coord[::step,2], c=Finit[::step], s=2, 
                vmin=0, vmax=.75, cmap=cm)
x_limits, y_limits, z_limits = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
x_range, x_middle = abs(x_limits[1] - x_limits[0]), np.mean(x_limits)
y_range, y_middle = abs(y_limits[1] - y_limits[0]), np.mean(y_limits)
z_range, z_middle = abs(z_limits[1] - z_limits[0]), np.mean(z_limits)
plot_radius = 0.5*max([x_range, y_range, z_range])
ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
plt.title('Function at epoch 0')
plt.savefig('d3sinit.png')

<IPython.core.display.Javascript object>

In [106]:
l = coord.shape[0]

x = tf.constant(coord[:,1], shape=[l,1])
y = tf.constant(coord[:,2], shape=[l,1])
z = tf.constant(coord[:,0], shape=[l,1])

p_ = tf.constant([0.5, 0.5], shape=[2, 1])

Finit_ = tf.concat([x,y],axis=1)
Finit_1 = tf.concat([x,y],axis=1)
Finit_2 = tf.concat([x,z],axis=1)


"""Finit_ = tf.random.uniform(
    shape=[4706,2],
    minval=0,
    maxval=None,
    dtype=tf.dtypes.float32,
    seed=None,
    name=None
)"""

'Finit_ = tf.random.uniform(\n    shape=[4706,2],\n    minval=0,\n    maxval=None,\n    dtype=tf.dtypes.float32,\n    seed=None,\n    name=None\n)'

In [311]:
F = tf.Variable(initial_value=np.array(Finit_, dtype=np.float32), trainable=False)
F1 = tf.Variable(initial_value=np.array(Finit_1, dtype=np.float32), trainable=False)
F2 = tf.Variable(initial_value=np.array(Finit_2, dtype=np.float32), trainable=False)
p = tf.Variable(initial_value=np.array(p_, dtype=np.float32), trainable=True)
model = SimplexTreeModel_ISM(p, stbase='data/human_simplextree.txt', dim=0, card1=50, card2=50)
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-1, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms = [], []
alpha, gamma = 1., .001
for epoch in tqdm(range(100+1)):
    
    with tf.GradientTape() as tape:
        
        #dgm = model.call()
        #loss = alpha * tf.square(wasserstein_distance(dgm, tf.constant(np.array([[-.98,-.03]], dtype=np.float32)), order=2, enable_autodiff=True))               
        dgm1, dgm2 = model.call()
        loss = alpha * tf.square(wasserstein_distance(dgm1, dgm2, order=2, enable_autodiff=True)) 

    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    #gradients = [tf.convert_to_tensor(gradients[0]) + np.random.normal(loc=0., scale=sigma, size=gradients[0].dense_shape)]
    gradients = [tf.convert_to_tensor(gradients[0])]
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append((dgm1,dgm2))

[autoreload of matplotlib.cbook.deprecation failed: Traceback (most recent call last):
  File "/Users/nyckeesluca/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/nyckeesluca/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/nyckeesluca/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/nyckeesluca/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/Users/nyckeesluca/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __class__ assignment: 'MatplotlibDeprecationWarning' object layout differs from 'MatplotlibDeprecati

In [63]:
import plotly.express as px
import plotly.graph_objects as go

In [310]:
fig = go.Figure()

fig.add_scatter(x=list(range(101)),y=losses, mode="lines", )

fig.update_layout(
    title="Loss curve - simplicial complex experiment",
    xaxis_title="Epochs",
    yaxis_title="Validation loss",
    legend_title="",
    height=350,
    width=500
)

fig.show()

### lower-star filtration on cubical complex

In [164]:
use_reg = 1

In [283]:
I = np.array(pd.read_csv('data/mnist_test.csv', header=None, sep=','), dtype=np.float32)
idx = np.argwhere(I[:,0] == 8)
image = np.reshape(-I[idx[8],1:], [28,28])
image = (image-image.min())/(image.max()-image.min())
image_clean = np.array(image)
image[2:5,2:5]        -= 0.6
image[25:27,25:27]    -= 0.6
image[25:27,2:5]      -= 0.6
image[1:4,24:26]      -= 0.6

To a tensor $X\in\mathbb{R}^{n\times n\times 2}$ (an image with pixels having values in $\mathbb{R}^2$), and a projection $p:\mathbb{R}^2\rightarrow\mathbb{R}$, one can associate a projected image $X_p\in\mathbb{n\times n}$ and compute the cubical persistence (with lower-star filtration) $$\mathrm{Pers}_{\mathrm{Cub}}(X_p).$$ 
Concretely, one has $(X_p)_{ij}=\sum_{k=1}^2p_k X_{ijk}$.

In [285]:
I_ = np.array([image,image]).reshape(28,28,2)
J_ = np.array([image_clean,image_clean]).reshape(28,28,2)

In [246]:
image_ = tf.reshape(tf.concat([image,image], 0),shape=[28,28,2])
p_ = tf.constant([0.5, 0.5], shape=[2, 1])

In [278]:
tf.reshape(tf.tensordot(image_,p_,1),shape=[28,28]).shape

TensorShape([28, 28])

In [315]:
X = tf.Variable(initial_value=np.array(I_, dtype=np.float32), trainable=False)
Y = tf.Variable(initial_value=np.array(J_, dtype=np.float32), trainable=False)
p = tf.Variable(initial_value=np.array(p_, dtype=np.float32), trainable=True)

model = CubicalModel_ISM(p, dim=0, card=256)

In [319]:
lr = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=1e-3, decay_steps=10, decay_rate=.01)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
sigma = 0.001

losses, dgms, empty = [], [], np.empty([0,2])
alpha = 10.
gamma = 1. if use_reg else 0
for epoch in tqdm(range(20+1)):
    
    with tf.GradientTape() as tape:
        
        dgm1, dgm2 = model.call()
        loss = alpha * tf.square(wasserstein_distance(dgm1, dgm2, order=2, enable_autodiff=True))
        
    gradients = tape.gradient(loss, model.trainable_variables)
    np.random.seed(epoch)
    gradients[0] = gradients[0] + np.random.normal(loc=0., scale=sigma, size=gradients[0].shape)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    losses.append(loss.numpy())
    dgms.append(dgm)
        

100%|██████████| 21/21 [00:13<00:00,  1.61it/s]


In [320]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(101)),y=losses,mode="lines"))
fig.update_layout(
    title="Loss curve - cubical complex experiment",
    xaxis_title="Epochs",
    yaxis_title="Validation loss",
    legend_title="",
    height=350,
    width=500
)
fig.show()

## tensorflow models 

### lower-star filtration on simplicial complex

In [130]:
#########################################
# Lower star filtration on simplex tree #
#########################################

# The parameters of the model are the vertex function values of the simplex tree.

def SimplexTree(stbase, fct, dim, card):
    # Parameters: stbase (array containing the name of the file where the simplex tree is located)
    #             fct (function values on the vertices of stbase),
    #             dim (homological dimension),
    #             card (number of persistence diagram points, sorted by distance-to-diagonal)
    
    # Copy stbase in another simplex tree st
    st = gd.SimplexTree()
    f = open(stbase[0], "r")
    for line in f:
        ints = line.split(" ")
        s = [int(v) for v in ints[:-1]]
        st.insert(s, -1e10)
    f.close()
        
    # Assign new filtration values
    for i in range(st.num_vertices()):
        st.assign_filtration([i], fct[i])
    st.make_filtration_non_decreasing()
    
    # Compute persistence diagram
    dgm = st.persistence()
    
    # Get vertex pairs for optimization. First, get all simplex pairs
    pairs = st.persistence_pairs()
    
    # Then, loop over all simplex pairs
    indices, pers = [], []
    for s1, s2 in pairs:
        # Select pairs with good homological dimension and finite lifetime
        if len(s1) == dim+1 and len(s2) > 0:
            # Get IDs of the vertices corresponding to the filtration values of the simplices
            l1, l2 = np.array(s1), np.array(s2)
            i1 = l1[np.argmax(fct[l1])]
            i2 = l2[np.argmax(fct[l2])]
            indices.append(i1)
            indices.append(i2)
            # Compute lifetime
            pers.append(st.filtration(s2) - st.filtration(s1))
    
    # Sort vertex pairs wrt lifetime
    perm = np.argsort(pers)
    indices = list(np.reshape(indices, [-1,2])[perm][::-1,:].flatten())
    
    # Pad vertex pairs
    indices = indices[:2*card] + [0 for _ in range(0,max(0,2*card-len(indices)))]
    return list(np.array(indices, dtype=np.int32))

class SimplexTreeModel_luca(tf.keras.Model):

    def __init__(self, p, stbase="simplextree.txt", dim=0, card=50):
        super(SimplexTreeModel_luca, self).__init__()
        self.p = p
        self.dim = dim
        self.card = card
        self.st = stbase
        
    def call(self):
        d, c = self.dim, self.card
        st = self.st

        fct = tf.linalg.matmul(F,self.p)

        # Turn STPers into a numpy function
        SimplexTreeTF = lambda fct: tf.numpy_function(SimplexTree, [np.array([st], dtype=str), fct, d, c], [tf.int32 for _ in range(2*c)])
        
        # Don't try to compute gradients for the vertex pairs
        fcts = tf.reshape(fct, [1, fct.shape[0]])
        inds = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(SimplexTreeTF, 
                                                                 fcts, dtype=[tf.int32 for _ in range(2*c)]))
        
        # Get persistence diagram
        self.dgm = tf.reshape(tf.gather_nd(fct, inds), [c,2]) 
        return self.dgm

class SimplexTreeModel_ISM(tf.keras.Model):

    def __init__(self, p, stbase="simplextree.txt", dim=0, card1=50, card2=50):
        super(SimplexTreeModel_ISM, self).__init__()
        self.p = p
        self.dim = dim
        self.card1 = card1
        self.card2 = card2
        self.st = stbase
        
    def call(self):
        d, c1, c2 = self.dim, self.card1, self.card2
        st = self.st

        fct1 = tf.linalg.matmul(F1,self.p)
        fct2 = tf.linalg.matmul(F2,self.p)

        # Turn STPers into a numpy function
        SimplexTreeTF1 = lambda fct: tf.numpy_function(SimplexTree, [np.array([st], dtype=str), fct1, d, c1], [tf.int32 for _ in range(2*c1)])
        SimplexTreeTF2 = lambda fct: tf.numpy_function(SimplexTree, [np.array([st], dtype=str), fct2, d, c2], [tf.int32 for _ in range(2*c2)])
        
        # Don't try to compute gradients for the vertex pairs
        fcts1 = tf.reshape(fct1, [1, fct1.shape[0]])
        inds1 = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(SimplexTreeTF1, 
                                                                 fcts1, dtype=[tf.int32 for _ in range(2*c1)]))

        fcts2 = tf.reshape(fct2, [1, fct2.shape[0]])
        inds2 = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(SimplexTreeTF2, 
                                                                 fcts2, dtype=[tf.int32 for _ in range(2*c2)]))                                                         
        
        # Get persistence diagram
        self.dgm1 = tf.reshape(tf.gather_nd(fct1, inds1), [c1,2])
        self.dgm2 = tf.reshape(tf.gather_nd(fct2, inds2), [c2,2])
        return self.dgm1, self.dgm2

### filtration on cubical complex

In [288]:
######################
# Cubical filtration #
######################

# The parameters of the model are the pixel values.

def Cubical(X, dim, card):
    # Parameters: X (image),
    #             dim (homological dimension), 
    #             card (number of persistence diagram points, sorted by distance-to-diagonal)

    # Compute the persistence pairs with Gudhi
    cc = gd.CubicalComplex(dimensions=X.shape, top_dimensional_cells=X.flatten())
    cc.persistence()
    try:
        cof = cc.cofaces_of_persistence_pairs()[0][dim]
    except IndexError:
        cof = np.array([])
        
    Xs = X.shape

    if len(cof) > 0:
        # Sort points with distance-to-diagonal
        pers = [X[np.unravel_index(cof[idx,1], Xs)] - X[np.unravel_index(cof[idx,0], Xs)] for idx in range(len(cof))]
        perm = np.argsort(pers)
        cof = cof[perm[::-1]]
    
    # Retrieve and ouput image indices/pixels corresponding to positive and negative simplices
    D = len(Xs)
    ocof = np.array([0 for _ in range(D*card*2)])
    count = 0
    for idx in range(0,min(2*card, 2*cof.shape[0]),2):
        ocof[D*idx:D*(idx+1)]     = np.unravel_index(cof[count,0], Xs)
        ocof[D*(idx+1):D*(idx+2)] = np.unravel_index(cof[count,1], Xs)
        count += 1
    return list(np.array(ocof, dtype=np.int32))

class CubicalModel_luca(tf.keras.Model):
    def __init__(self, p, dim=1, card=50):
        super(CubicalModel_luca, self).__init__()
        self.p = p
        self.dim = dim
        self.card = card
        
    def call(self):

        Xp = tf.reshape(tf.tensordot(X,p,1),shape=[28,28])

        d, c, D = self.dim, self.card, len(Xp.shape)
        XX = tf.reshape(Xp, [1, Xp.shape[0], Xp.shape[1]])
        
        # Turn numpy function into tensorflow function
        CbTF = lambda Xp: tf.numpy_function(Cubical, [Xp, d, c], [tf.int32 for _ in range(2*D*c)])
        
        
        # Compute pixels associated to positive and negative simplices 
        # Don't compute gradient for this operation
        A = tf.map_fn(CbTF,XX,fn_output_signature=[tf.int32 for _ in range(2*D*c)])
        inds = tf.nest.map_structure(tf.stop_gradient, A)
        
        # Get persistence diagram by simply picking the corresponding entries in the image
        dgm = tf.reshape(tf.gather_nd(Xp, tf.reshape(inds, [-1,D])), [-1,2])
        return dgm

class CubicalModel_ISM(tf.keras.Model):
    def __init__(self, p, dim=1, card=50):
        super(CubicalModel_ISM, self).__init__()
        self.p = p
        self.dim = dim
        self.card = card
        
    def call(self):

        Xp = tf.reshape(tf.tensordot(X,p,1),shape=[28,28])
        Yp = tf.reshape(tf.tensordot(Y,p,1),shape=[28,28])

        d, c, D = self.dim, self.card, len(Xp.shape)
        XX = tf.reshape(Xp, [1, Xp.shape[0], Xp.shape[1]])
        YY = tf.reshape(Yp, [1, Yp.shape[0], Yp.shape[1]])
        
        # Turn numpy function into tensorflow function
        CbTF1 = lambda Xp: tf.numpy_function(Cubical, [Xp, d, c], [tf.int32 for _ in range(2*D*c)])
        CbTF2 = lambda Yp: tf.numpy_function(Cubical, [Yp, d, c], [tf.int32 for _ in range(2*D*c)])
        
        # Compute pixels associated to positive and negative simplices 
        # Don't compute gradient for this operation
        
        inds1 = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(CbTF1,XX,fn_output_signature=[tf.int32 for _ in range(2*D*c)]))
        inds2 = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(CbTF2,YY,fn_output_signature=[tf.int32 for _ in range(2*D*c)]))

        
        # Get persistence diagram by simply picking the corresponding entries in the image
        dgm1 = tf.reshape(tf.gather_nd(Xp, tf.reshape(inds1, [-1,D])), [-1,2])
        dgm2 = tf.reshape(tf.gather_nd(Yp, tf.reshape(inds2, [-1,D])), [-1,2])
        return dgm1, dgm2